# <center> <b> <h1>   2. SVM RFE (ranking) </h1></b></center>

In [1]:
import joblib
from sklearn.svm import SVC
from sklearn.feature_selection import RFECV

import pandas as pd

PATH_PKL = "../1. SVM RFE (All features)/output/"
PATH_DATASETS = "../../2_Data_preparation/2_Split_Raw_Datasets/output/"

In [2]:
def readCsvs(split):
    datasets = []
    constant1 = 'X_'
    constant2 = '_' + split
    features = ['area', 'meancurv', 'thickness', 'volume']

    for feature in features:
        df = pd.read_csv(PATH_DATASETS + constant1 + feature + constant2 +'.csv', index_col=0)
        datasets.append({'name': feature, 'data': df})
    return datasets

In [3]:
def readClf():
    clasificators = []
    constant1 = "SVM RFE (AllFeatures) "
    features = ['area', 'meancurv', 'thickness', 'volume']

    for feature in features:
        clf = joblib.load(PATH_PKL + constant1 + feature + '.pkl')
        clasificators.append(clf)
        
    return clasificators

# 1. Prepare the Data

In [4]:
df_X_tr = readCsvs('train')
df_X_tr[0]['data'].shape

(550, 148)

In [5]:
clfs = readClf()
clfs[0]

{'name': 'area',
 'rfecv': RFECV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                     decision_function_shape='ovr', degree=3,
                     gamma='auto_deprecated', kernel='linear', max_iter=1000000,
                     probability=False, random_state=None, shrinking=True,
                     tol=0.001, verbose=False),
       min_features_to_select=1, n_jobs=-1, scoring='accuracy', step=1,
       verbose=True),
 'kernel': 'linear'}

# 2. Identify the most important feature of the SVM RFE

In [ ]:
rankings =[]
# print the name
for feature, rfcv in zip(df_X_tr, clfs):
    df_fetures = pd.DataFrame(columns=['Name','Ranking', 'Support'])
    df_fetures.set_index('Name', inplace = True)
    name = feature['name']
    data = feature['data']
    rfcv_data = rfcv['rfecv']
    #df_fetures.head()
    for roi in zip(data.columns.values, rfcv_data.support_, rfcv_data.ranking_):
        print(roi)
        df_fetures.at[roi[0],"Support"] = roi[1]
        df_fetures.at[roi[0],"Ranking"] = roi[2]
    rankings.append({'name' : name, 'data' : df_fetures})
    print("end with: " + name + '\n')

In [8]:
for ranking in rankings:
    data = ranking['data']
    data.sort_values("Ranking", axis=0, ascending= True, inplace= True)

rankings[0]['data']

,Ranking,Support
Name,,
lh_S_circular_insula_inf_area,1,True
lh_G_and_S_cingul-Mid-Ant_area,1,True
lh_G_oc-temp_med-Parahip_area,1,True
rh_S_front_inf_area,2,False
lh_S_oc_middle_and_Lunatus_area,3,False
...,...,...
lh_G_front_middle_area,142,False
rh_S_intrapariet_and_P_trans_area,143,False
lh_G_cingul-Post-ventral_area,144,False


# Save the Rankings on CSVs

In [8]:
for ranking in rankings:
    data = ranking['data']
    name = ranking['name']
    data.to_csv("output/SVMRFE (Support)" + name + ".csv")
    data[data["Support"]==1].to_csv("output/SVMRFE (Support - 1)" + name + ".csv")